<p><img style="padding: 0 15px; float: left;" src="Images/FTLogo300.jpg" alt="FT Crusader Logo" Width='140' Height= '250'/></p> 
<p> <h2> Frogtown DataLoad Workbook, 05/06/18</h2> <a name="tc"></a>
<p> <h3> By Frogtown Crusader (Abu Nayeem)</h3>

This is just clean coded process going over the uploading steps assuming all the preliminary steps after decoder are made


## Table of contents <a name="content"></a>
* [Data Setup](#setup)
* [Create Variables](#create)
* [Intersection Table: Preparation](#intersection)
* [Address Table: Preparation](#address)
* [Preliminary Data Cleaning](#pre)


### Data Setup <a name="setup"></a>

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline 
from IPython.display import HTML
from IPython.display import display
import requests # library to handle requests

#Load Data
#df_crime = pd.read_csv('Datasets/Crime_Incident_Report_-_Dataset.csv')

#rename columns
#cols= ['Case','Date','Time','Code','IncType','Incident','Grid','NNum','Neighborhood','Block','CallDispCode','CallDisposition', 'Count']
#df_crime.columns= cols

#selection for Frogtown and nearby area
#df=df_crime.query('Grid in [66,67, 68, 86, 87,88,89, 90, 91, 92,106,107,108,109,110]')

#Socarata upload Method
from sodapy import Socrata

#New Upload Method Get Information from Socrata API
client = Socrata("information.stpaul.gov", None)

#Easier to bulk upload
results = client.get("gppb-g9cg", limit=1000000)
df = pd.DataFrame.from_records(results)
# Find Max Date Value
results = client.get("gppb-g9cg", limit=1)
r_max = pd.DataFrame.from_records(results)

#rename columns [Note the order of Columns have changed]
cols= ['Block','CallDispCode','CallDisposition','Case','Code', 'Count','Date','Grid','Incident','IncType','Neighborhood','NNum','Time']
df.columns= cols
df = df.astype({"Case": int, "Code": int, "Grid":float, "NNum":int,"Count":int})
#select respective Grids of interest
df=df.query("Grid in (66,67, 68, 86, 87,88,89, 90, 91, 92,106,107,108,109,110)")

df.head(4)

,Block,CallDispCode,CallDisposition,Case,Code,Count,Date,Grid,Incident,IncType,Neighborhood,NNum,Time
0,55X CENTRAL AV W,A,Advised,19205620,2619,1,2019-09-11T23:44:14.000,109.0,Discharge,"Weapons, Discharging a Firearm in the City Limits",NaN,8,2019-09-11T23:44:14.000
9,73X THOMAS AV,G,Gone on Arrival,19205595,600,1,2019-09-11T23:01:50.000,88.0,Theft,"Theft, Except Auto Theft",NaN,7,2019-09-11T23:01:50.000
29,UNIVERSITY AV W & DALE,A,Advised,19205539,9954,1,2019-09-11T21:32:59.000,89.0,Proactive Police Visit,Proactive Police Visit,NaN,7,2019-09-11T21:32:59.000
31,19X EDMUND AV,RR,Report Written,19205533,1811,1,2019-09-11T21:26:00.000,90.0,Narcotics,"Narcotics, Possession of Cocaine",NaN,7,2019-09-11T21:26:00.000


### Create New Variables <a name="create"></a>

In [2]:
#Add Time Variables
df= df[df.Case != 18254093] #messed up time variable

#Convert Date to Datetime!
from datetime import datetime

df['DateTime']= pd.to_datetime(df['Date']) # Create new column called DateTime
df['Year']= df['DateTime'].dt.year #create year column
df['DayofWeek']=df['DateTime'].dt.dayofweek #create day of the week column where default 0=Monday
df['Weekend'] = df['DayofWeek'].apply(lambda x: 1 if (x>4)  else 0) #Create a weekend category
df['Month'] = df['DateTime'].dt.month # Create Month Category
df['Day'] = df['DateTime'].dt.day #Create Day of the Current month
df['DayYear'] = df['DateTime'].dt.dayofyear  #Create Day of the year (0-365)
df['Day_Max'] = df.iloc[0,-1] #selects uptodate day; NOTE: the data is sorted chronologically

#Hour Data
df['TimeHour']= pd.to_datetime(df['Time'])
df['Hour'] = df['TimeHour'].dt.hour.astype(int) #Create Hour Colum
df['LateNight'] = df['Hour'].apply(lambda x: 1 if (x>21 or x<5)  else 0) #Latenight designation from 10Pm to 6PM

#Creating the intersection Column. Note: the Block column has the address information
df.Block = df.Block.astype(str) #first change the type to string
df['Block']= df['Block'].str.lower() #lowercase string to create uniformity

#While scanning the data I noticed that all intersections had "&" 
df['Intersection'] = df['Block'].apply(lambda x: 1 if '&' in x else 0) #intersection

df.head(5)

,Block,CallDispCode,CallDisposition,Case,Code,Count,Date,Grid,Incident,IncType,...,DayofWeek,Weekend,Month,Day,DayYear,Day_Max,TimeHour,Hour,LateNight,Intersection
0,55x central av w,A,Advised,19205620,2619,1,2019-09-11T23:44:14.000,109.0,Discharge,"Weapons, Discharging a Firearm in the City Limits",...,2,0,9,11,254,254,2019-09-11 23:44:14,23,1,0
9,73x thomas av,G,Gone on Arrival,19205595,600,1,2019-09-11T23:01:50.000,88.0,Theft,"Theft, Except Auto Theft",...,2,0,9,11,254,254,2019-09-11 23:01:50,23,1,0
29,university av w & dale,A,Advised,19205539,9954,1,2019-09-11T21:32:59.000,89.0,Proactive Police Visit,Proactive Police Visit,...,2,0,9,11,254,254,2019-09-11 21:32:59,21,0,1
31,19x edmund av,RR,Report Written,19205533,1811,1,2019-09-11T21:26:00.000,90.0,Narcotics,"Narcotics, Possession of Cocaine",...,2,0,9,11,254,254,2019-09-11 21:26:00,21,0,0
35,university av w & lexington,A,Advised,19205522,9954,1,2019-09-11T21:09:41.000,87.0,Proactive Police Visit,Proactive Police Visit,...,2,0,9,11,254,254,2019-09-11 21:09:41,21,0,1


### Prepare Intersection table <a name="intersection"></a>

In [3]:
#Load clean intersections key
df_Intersection_key = pd.read_csv('Datasets/Intersection_key_clean.csv')

# Create a new dateframe specifying only intersections from primary dataset
dfI=df.query('Intersection ==1')
print('The intersection table dimension are ' + str(dfI.shape))
#print(dfI.Block.head(10))


#Split the strings
new=dfI['Block'].str.split("& ", n = 1, expand = True) 
dfI['Inter2']= new[1]
new=dfI['Block'].str.split(" ", n = 1, expand = True) #Note the code specifies the first time a space occured
dfI['Inter1']=new[0]

#Create the IndexKey; recall we prepared the IntersectionKey where it considers any order
dfI['IndexKey']= dfI['Inter1']+ '_' + dfI['Inter2']
dfI.reset_index()
dfI=pd.merge(dfI, df_Intersection_key, on='IndexKey', how='left')



The intersection table dimension are (6079, 25)


In [4]:
#Drop rows with missing coordinates
dfI=dfI[dfI['Coordinates'].notnull()]

# Separate Latitude and Longitude 
new=dfI['Coordinates'].str.split(",", n = 1, expand = True) 
# making seperate first name column from new data frame 
dfI['Latitude']= pd.to_numeric(new[0]) #pd.to_numeric convert it to float
dfI['Longitude']= pd.to_numeric(new[1])

#Renaming columns

dfI['Block']=dfI['OutputKey'] #for practical purposes it makes sense
Drop_col=['Inter2','Inter1', 'IndexKey', 'Coordinates', 'OutputKey']
dfI_Final=dfI.drop(Drop_col, axis=1,)
dfI_Final.head(5)


,Block,CallDispCode,CallDisposition,Case,Code,Count,Date,Grid,Incident,IncType,...,Month,Day,DayYear,Day_Max,TimeHour,Hour,LateNight,Intersection,Latitude,Longitude
0,dale_university,A,Advised,19205539,9954,1,2019-09-11T21:32:59.000,89.0,Proactive Police Visit,Proactive Police Visit,...,9,11,254,254,2019-09-11 21:32:59,21,0,1,44.955828,-93.126376
1,lexington_university,A,Advised,19205522,9954,1,2019-09-11T21:09:41.000,87.0,Proactive Police Visit,Proactive Police Visit,...,9,11,254,254,2019-09-11 21:09:41,21,0,1,44.955826,-93.146539
3,grotto_thomas,A,Advised,19205209,9954,1,2019-09-11T15:19:00.000,88.0,Proactive Police Visit,Proactive Police Visit,...,9,11,254,254,2019-09-11 15:19:00,15,0,1,44.959356,-93.131431
4,dale_university,A,Advised,19205099,9959,1,2019-09-11T13:08:00.000,89.0,Community Engagement Event,Community Engagement Event,...,9,11,254,254,2019-09-11 13:08:00,13,0,1,44.955828,-93.126376
5,chatsworth_thomas,A,Advised,19204779,9954,1,2019-09-11T01:32:21.000,87.0,Proactive Police Visit,Proactive Police Visit,...,9,11,254,254,2019-09-11 01:32:21,1,1,1,44.959365,-93.141564


### Prepare Address Decoder Table <a name="address"></a>

It is so remarkably short

In [5]:
#Split Data
dfW=df.query('Intersection==0')

#Load Complete Decoder Key
df_C= pd.read_csv('Datasets/SemiKey.csv')
df_C= df_C[['Block','Latitude','Longitude']]

# Merge with the dataset and remove missing values
dC=pd.merge(dfW, df_C, on='Block', how='left')
dC=dC.fillna('Mi')
dC=dC.query('Latitude != "Mi"')

#Bringing the data back together 
fg= dfI.append(dC, ignore_index=True)

### Preliminary Pre-Cleaning steps <a name="clean"></a>

It is good to do the data pre-cleaning steps here to reduce clutter on a working notebook. A few edits include renaming some values, clustering crimes together and creating some dummies. It is saved in a csv, which is used for execution

In [6]:
#Few Quick Edits
fg.CallDisposition.loc[(fg['CallDisposition'] == 'G - Gone on Arrival')] = 'Gone on Arrival'
fg.CallDisposition.loc[(fg['CallDisposition'] == 'A - Advised')] = 'Advised'
fg.CallDisposition.loc[(fg['CallDisposition'] == 'RR - Report Written')] = 'Report Written'
fg.Incident.loc[(fg['Incident'] == 'Simple Asasult Dom.')] = 'Simple Assault Dom.'
fg.Incident.loc[(fg['Incident'] == 'Graffiti')] = 'Vandalism'
fg.Incident.loc[fg["Incident"].isin([ "Rape","Agg. Assault",'Homicide'])]= 'Violent'
fg.Incident.loc[fg["Incident"].isin(["Simple Assault Dom.","Agg. Assault Dom."])]= 'Domestic Assault'

#[fg["Incident"].isin(["Simple Assault Dom.", "Rape"])


#Create a dummy for each crime category
fg= pd.concat([fg,pd.get_dummies(fg['Incident'])], axis=1)
fg= pd.concat([fg,pd.get_dummies(fg['CallDisposition'])], axis=1)

fg.to_csv('Datasets/FGCrime_Final.csv', encoding='utf-8', index=False)

In [9]:
fg.head(4)

,Block,CallDispCode,CallDisposition,Case,Code,Coordinates,Count,Date,DateTime,Day,...,Domestic Assault,Narcotics,Proactive Police Visit,Robbery,Theft,Vandalism,Violent,Advised,Gone on Arrival,Report Written
0,western_thomas,A,Advised,19199810,9954,"44.959379, -93.116175",1,2019-09-04T22:22:17.000,2019-09-04 22:22:17,4,...,0,0,1,0,0,0,0,1,0,0
1,western_charles,A,Advised,19199677,9954,"44.957571, -93.116178",1,2019-09-04T19:14:58.000,2019-09-04 19:14:58,4,...,0,0,1,0,0,0,0,1,0,0
2,dale_university,A,Advised,19199566,9954,"44.955828, -93.126376",1,2019-09-04T17:16:56.000,2019-09-04 17:16:56,4,...,0,0,1,0,0,0,0,1,0,0
3,hamline_university,G,Gone on Arrival,19199320,9954,"44.955811, -93.156832",1,2019-09-04T13:06:54.000,2019-09-04 13:06:54,4,...,0,0,1,0,0,0,0,0,1,0


In [ ]:
Ward1.query('ElectionDescription in ("11/08/2016 - STATE GENERAL", "08/09/2016 - STATE PRIMARY","11/07/2017 - MUNICIPAL GENERAL","11/07/2017 - SCHOOL DISTRICT GENERAL"  ,"11/08/2016 - STATE GENERAL","08/09/2016 - STATE PRIMARY","11/03/2015 - MUNICIPAL GENERAL","11/03/2015 - SCHOOL DISTRICT GENERAL","11/08/2011 - MUNICIPAL GENERAL","11/06/2018 - STATE GENERAL","08/14/2018 - STATE PRIMARY")' 